## Projet CNN

In [ ]:
# Environnement de Kaggle qui permet de ne pas utiliser le cpu et la ram de notre machine et qui a deja plusieurs librairies pre-installés

import numpy as np 
import pandas as pd 
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils




import os
        
DIR_PATH = '/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database/'
CLASSES = ['COVID', 'NORMAL', 'Viral Pneumonia']

In [ ]:
all_data = []
for dirname, _, filenames in os.walk(DIR_PATH): # os.walk retourne premierement le Path puis les directories sous le path puis tous les fichiers du path donc dirname ici sont covid,normal et vp et filenames sont les images sous chaque dirname.
    for file in filenames:
        if 'png' in file:
            image = cv2.imread(os.path.join(dirname, file))
            image = cv2.resize(image, (64, 64)) / 255.0 # we resize the image to 64 height and 64 width, we divide by 255 to obtain normalized values between 0 and 1(bcs pixels of the image are between 0 and 255), so that the optimizer doesnt explode and be efficient
            label = CLASSES.index(os.path.split(dirname)[-1]) # label sera 0 ou 1 ou 2 un int donc
            #print(os.path.split(dirname)[-1]) : retourne '/kaggle/input/covid19-radiography-database/COVID-19 Radiography Database', 'NORMAL' en premier donc le -1 est pour prendre le dernier élément de ce retour qui est 'NORMAL'
            all_data.append([image, label])
            

In [ ]:
len(all_data)

In [ ]:
X = []
y = []
for img, lbl in all_data:
    X.append(img)
    y.append(lbl)
X = np.array(X)
y = np.array(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train.shape,X_test.shape

## Premier modèle avec 9 couches

In [ ]:
# input shape de l'image (64,64,3) :(width,height,RGB channels),on a le 3 car images couleurs
model_1 = models.Sequential() # Le model est sequentiel c'est a dire les données rentrent batch par batch sequence par sequence 
    
model_1.add(layers.Conv2D(30, (5, 5), input_shape=(64, 64, 3), activation='relu')) # Relu fonction linéaire donc elle permet de converger plus rapidement,on sélectionne 30 features de l'image,notamment détection des edges avec une matrice 5*5(filtrage)
model_1.add(layers.MaxPooling2D(pool_size=(2, 2))) # Reduction de la dimension de l'image par le biais d'une matrice 2*2 où on garde le max
    
model_1.add(layers.Conv2D(15, (3, 3), activation='relu')) # on sélectionne 15 features de l'image,avec une matrice 3*3 (filtrage)
model_1.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
model_1.add(layers.Dropout(0.2)) # on perd 20% des neurones du réseau afin d'éviter un over-fitting.
model_1.add(layers.Flatten()) # transforme en image qui a une shape à 3 dimensions en un vecteur 1 dimension
    
model_1.add(layers.Dense(128, activation='relu')) # couche avec 128 neuronnes
model_1.add(layers.Dense(50, activation='relu')) # couche avec 50 neuronnes
model_1.add(layers.Dense(3, activation='softmax')) # on utilise la fonction softmax car elle permet d'avoir des valeurs en output entre [0,1] ce qui permet une classification avec n'importe quel nbr de classes ici on en a plus que 2 (cas non binaires)
    
# Compile model
model_1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
#Adam optimizer pour optimiser nos poids ,SparseCategoricalCrossentropyloss function when there are two or more label classes. We expect labels to be provided as integers, which is our case here.

In [ ]:
history_1 = model_1.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_test, y_test),batch_size=50)

In [ ]:
scores_1 = model_1.evaluate(X_test, y_test, verbose=0)
print('Final CNN accuracy of model 1: ', scores_1[1]*100, "%")

## Deuxième modèle avec 7 couches

In [ ]:
#Initializing the CNN
#There is also a graph option but we'll use sequential ANN Model
model_2 = models.Sequential()

#step 1 - Convolution
#creating the feature map by using feature detector from ınput image

model_2.add(layers.Convolution2D(32,3,3, input_shape=(64,64,3), activation='relu'))
#32 Feature maps&detetctors uses 3 by 3 matrices, we can put 128 in the powerful machines

#step -2 Pooling
model_2.add(layers.MaxPooling2D(pool_size = (2,2)))

#second convolution and pooling steps.
model_2.add(layers.Convolution2D(32,3,3, input_shape=(64,64,3), activation='relu'))

model_2.add(layers.MaxPooling2D(pool_size = (2,2)))

#step -3 Flattening
model_2.add(layers.Flatten())

#step-4 Full connection step
model_2.add(layers.Dense(64, activation = 'relu'))
model_2.add(layers.Dense(3, activation = 'softmax'))


model_2.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics = ['accuracy'])


In [ ]:
history_2 = model_2.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_test, y_test),batch_size=50)

In [ ]:
scores_2 = model_2.evaluate(X_test, y_test, verbose=0)
print('Final CNN accuracy of model 2: ', scores_2[1]*100, "%")

## On remarque donc que le premier modèle est le meilleur en terme d'accuracy , réssayons son apprentissage avec cette fois-ci sgd en optimizer

In [ ]:
# input shape de l'image (64,64,3) :(width,height,RGB channels),on a le 3 car images couleurs
model_1_sgd = models.Sequential()
    
model_1_sgd.add(layers.Conv2D(30, (5, 5), input_shape=(64, 64, 3), activation='relu')) # Relu fonction linéaire donc elle permet de converger plus rapidement,on sélectionne 30 features de l'image,notamment détection des edges avec une matrice 5*5(filtrage)
model_1_sgd.add(layers.MaxPooling2D(pool_size=(2, 2))) # Reduction de la dimension de l'image par le biais d'une matrice 2*2 où on garde le max
    
model_1_sgd.add(layers.Conv2D(15, (3, 3), activation='relu')) # on sélectionne 15 features de l'image,avec une matrice 3*3 (filtrage)
model_1_sgd.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
model_1_sgd.add(layers.Dropout(0.2)) # on perd 20% des neurones du réseau afin d'éviter un over-fitting.
model_1_sgd.add(layers.Flatten()) # transforme en image qui a une shape à 3 dimensions en un vecteur 1 dimension
    
model_1_sgd.add(layers.Dense(128, activation='relu')) # couche avec 128 neuronnes
model_1_sgd.add(layers.Dense(50, activation='relu')) # couche avec 50 neuronnes
model_1_sgd.add(layers.Dense(3, activation='softmax')) # on utilise la fonction softmax car elle permet d'avoir des valeurs en output entre [0,1] ce qui permet une classification avec n'importe quel nbr de classes ici on en a plus que 2 (cas non binaires)
    

model_1_sgd.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='sgd', metrics=['accuracy'])

In [ ]:
history_1_sgd = model_1_sgd.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_test, y_test),batch_size=50)

In [ ]:
scores_1_sgd = model_1_sgd.evaluate(X_test, y_test, verbose=0)
print('Final CNN accuracy of model 1 with sgd: ', scores_1_sgd[1]*100, "%")

## Comme on peut le voir notre premier modèle est  plus efficace avec adam en optimizer car l'optimizer adam a tendance a convergé plus rapidement que le sgd et vu qu'ici le nombre d'epochs n'est que de 30 alors adam est plus efficace. Mais en général et adam et sgd optimizer restent très populaires car ils peuvent êtres utilisés dans la plupart des cas.

 ## We plot the loss and accuracy of our first model over 30 epochs

In [ ]:
def plot_acc_loss(results, epochs):
 acc = results.history['accuracy']
 loss = results.history['loss']
 val_acc = results.history['val_accuracy']
 val_loss = results.history['val_loss']
 plt.figure(figsize=(15, 5))
 plt.subplot(121)
 plt.plot( acc[1:], label='Train_acc')
 plt.plot( val_acc[1:], label='Test_acc')
 plt.title('Accuracy over ' + str(epochs) + ' Epochs', size=15)
 plt.legend()
 plt.grid(True)
 plt.subplot(122)
 plt.plot( loss[1:], label='Train_loss')
 plt.plot( val_loss[1:], label='Test_loss')
 plt.title('Loss over ' + str(epochs) +  ' Epochs', size=15)
 plt.legend()
 plt.grid(True)
 plt.show()
 
plot_acc_loss(history_1, 30)

## We plot the loss and accuracy of our second model over 30 epochs

In [ ]:
plot_acc_loss(history_2, 30)

## We plot the loss and accuracy of our first model this time with sgd as optimizer over 30 epochs.

In [ ]:
plot_acc_loss(history_1_sgd, 30)

## This time we will hot-encode our output to use a different loss function with our first model

## Before we had labels as integers : 0 : covid , 1 : normal and 2 : viral pneumonia for example , now our labels will be vectors: (1,0,0) for example for covid. This will give us the possibility to use a different loss function and see the difference between having and int as outpout and a vector. Because sometimes using integers as output can give higher weights to the category with the higher number and this could mess with our model.

In [ ]:
# one hot encode outputs
y_train_cnn = np_utils.to_categorical(y_train)
y_test_cnn = np_utils.to_categorical(y_test)


In [ ]:
# input shape de l'image (64,64,3) :(width,height,RGB channels),on a le 3 car images couleurs
model_1_encode = models.Sequential()
    
model_1_encode.add(layers.Conv2D(30, (5, 5), input_shape=(64, 64, 3), activation='relu',)) # Relu fonction linéaire donc elle permet de converger plus rapidement,on sélectionne 30 features de l'image,notamment détection des edges avec une matrice 5*5(filtrage)
model_1_encode.add(layers.MaxPooling2D(pool_size=(2, 2))) # Reduction de la dimension de l'image par le biais d'une matrice 2*2 où on garde le max
    
model_1_encode.add(layers.Conv2D(15, (3, 3), activation='relu')) # on sélectionne 15 features de l'image,avec une matrice 3*3 (filtrage)
model_1_encode.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
model_1_encode.add(layers.Dropout(0.2)) # on perd 20% des neurones du réseau afin d'éviter un over-fitting.
model_1_encode.add(layers.Flatten()) # transforme en image qui a une shape à 3 dimensions en un vecteur 1 dimension
    
model_1_encode.add(layers.Dense(128, activation='relu')) # couche avec 128 neuronnes
model_1_encode.add(layers.Dense(50, activation='relu')) # couche avec 50 neuronnes
model_1_encode.add(layers.Dense(3, activation='softmax')) # on utilise la fonction softmax car elle permet d'avoir des valeurs en output entre [0,1] ce qui permet une classification avec n'importe quel nbr de classes ici on en a plus que 2 (cas non binaires)
    
# Compile model
model_1_encode.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#Adam optimizer pour optimiser nos poids ,CategoricalCrossentropyloss function when there are two or more label classes. We expect labels to be provided as vectors.

In [ ]:
history_1_encode = model_1_encode.fit(X_train, y_train_cnn, epochs=30, 
                    validation_data=(X_test, y_test_cnn))

In [ ]:
scores_1_encode = model_1_encode.evaluate(X_test, y_test_cnn, verbose=0)
print('Final CNN accuracy of model 1 with hot encoding: ', scores_1_encode[1]*100, "%")

In [ ]:
plot_acc_loss(history_1_encode, 30)

## Some predictions with external data

## First model with sgd optimizer

In [ ]:
from keras.preprocessing import image
test_image = image.load_img('/kaggle/input/test-cnn-covid/Covid19.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image) # converts image to array
test_image = np.expand_dims(test_image, axis = 0) # adds dimension because convolutional layer expects 4 dimensions (the added dimension is the batch size dimension to feed N number of data points at time to the model)
output = model_1_sgd.predict(test_image) # return vector with probabilities for each class
print("Predicted:", CLASSES[np.argmax(output)]) # max so that it takes the one which is the prediction and returns the name of the predicted class

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/covid19_2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1_sgd.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/healthy.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1_sgd.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/vp2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1_sgd.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

## Second model

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/Covid19.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_2.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/covid19_2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_2.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/healthy.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_2.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/healthy2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_2.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/healthy4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_2.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/vp2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_2.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

## First model with adam optimizer

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/Covid19.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/covid19_2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/healthy.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/vp2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

## Firs model with one hot encoding

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/Covid19.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1_encode.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/healthy.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
output = model_1_encode.predict(test_image)
print("Predicted:", CLASSES[np.argmax(output)])

In [ ]:
test_image = image.load_img('/kaggle/input/test-cnn-covid/vp2.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image) 
test_image = np.expand_dims(test_image, axis = 0) 
output = model_1_encode.predict(test_image) 
print("Predicted:", CLASSES[np.argmax(output)]) 

## We see that since the dataset is too small we can't really exploit the results of our models we will need way more images in order to avoid overfitting and to train our models properly, but in terms of predictions the second model is the most interesting one.